In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv") 

In [ ]:
movies.head

In [ ]:
credits.head

In [ ]:
movies = movies.merge(credits,on='title')

In [ ]:
movies.info()

In [ ]:
movies.head(1)

here budget,homepage,language,production companies,popularity,original title,spoken language ,tag line ,vote average and count,movie_id are removed

In [ ]:
movies = movies[["movie_id","title","overview","genres","keywords","cast","crew"]]

In [ ]:
movies.head()

In [ ]:
movies.shape

In [ ]:
credits.shape

In [ ]:
movies.dropna(inplace=True)


In [ ]:
movies.isnull().sum()

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.columns[0:]

In [ ]:
movies.iloc[0].genres

here we are getting them as string we need to convert thim  into list

we need to bring this into ["Action","Adventure","Fantasy","SCi-fi"] for

In [ ]:
import ast
def convert(obj):
    L =[]
    for i in ast.literal_eval(obj):
        L.append(i["name"])
    return L 

In [ ]:
movies["genres"]=movies["genres"].apply(convert)

In [ ]:
movies["keywords"]=movies["keywords"].apply(convert)

In [ ]:
movies.head()

In [ ]:

def convert3(obj):
    L =[]
    counter = 0

    for i in ast.literal_eval(obj):
        if counter!=3 :
            L.append(i["name"])
            counter+=1
        else:
            break
    return L 

In [ ]:
movies["cast"]=movies["cast"].apply(convert3)

In [ ]:
movies.head()

In [ ]:
movies.iloc[0].crew

In [ ]:
def convertder(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i["job"]=="Director":
            L.append(i["name"])
            break
    return L  

In [ ]:
movies['crew'] = movies["crew"].apply(convertder) 

In [ ]:
movies.head()

here overview is a string so we convert it to string and contatinate with other lists

In [ ]:
movies['overview']=movies["overview"].apply(lambda x:x.split())

In [ ]:
movies.head()

here we need to remove spaces in genre,cast,crew 
because for example Sam Worthington and Sam Mendes will become 4 different words and if we need to  watch movies of Sam Mendes model wii get cofused and also gives Sam Worthington movies

In [ ]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies["crew"]=movies["crew"].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies['tags']=movies['overview']+movies['genres']+movies['cast']+movies['crew']

In [ ]:
movies.head()

In [ ]:
new_df = movies[['movie_id','title','tags']]

In [ ]:
new_df["tags"] = new_df["tags"].apply(lambda x:" ".join(x))

In [ ]:
new_df.head()

In [ ]:
new_df["tags"][0]

In [ ]:
new_df["tags"] = new_df["tags"].apply(lambda x:x.lower())

In [ ]:
new_df["tags"][0]

now we need to vectorize these tags and recommend movies 

here we convert text to vectors and choose the other vectors that are nearer to our defined movie here we are using bag of words techique

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words="english")

In [ ]:
vectors = cv.fit_transform(new_df["tags"]).toarray()

In [ ]:
vectors

here there will be words like action,actions etc or acceots,accepted,accepting etc  so we apply stemming

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()


In [ ]:
#helper fuction
def stem(obj):
    y = []

    for i in obj.split():
        y.append(stemmer.stem(i))
    return " ".join(y)    

In [ ]:
new_df["tags"]=new_df["tags"].apply(stem)

In [ ]:
cv.get_feature_names_out(vectors)

here we are calculating the distances using cosine distances between anges that is based on the angle

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity 


In [ ]:
similarities = cosine_similarity(vectors)

In [ ]:
similarities.shape

In [ ]:
print(similarities)

In [ ]:
similarities[0]

this determines that first movie is 100% similar to first movie and 0.15% persent similar to second movie

when we try to sort the similarities we will loose the indices which is bad as we are working based on the  induices to identify the movise

In [94]:
sorted(list(enumerate(similarities[0])),reverse=True,key=lambda x:x[1])[1:6]

[(4803, 0.2631806779839076),
 (4802, 0.25197631533948484),
 (4801, 0.24313226954193232),
 (4800, 0.2380952380952381),
 (4799, 0.23570226039551584),
 (4798, 0.2182178902359924),
 (4797, 0.21821789023599236),
 (4796, 0.2152488010002526),
 (4795, 0.21398024625545647),
 (4792, 0.21170244960998527)]

In [97]:
def recommend(movie):
    movie_index = new_df[new_df['title']==movie].index[0]
    distances = similarities[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [102]:
recommend('Spider-Man 2')

Spider-Man 3
Spider-Man
The Amazing Spider-Man
The Amazing Spider-Man 2
Superman
Small Soldiers
Deadpool
In the Name of the King: A Dungeon Siege Tale
The Last Legion
Oz: The Great and Powerful
